In [22]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from geopy.distance import geodesic


In [3]:
path = '../data/'
file = 'airport-codes.csv'
df = pd.read_csv(path+file)
df 

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"40.07080078125, -74.93360137939453"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"38.704022, -101.473911"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"59.947733, -151.692524"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"34.86479949951172, -86.77030181884766"
4,00AN,small_airport,Katmai Lodge Airport,80.0,NaN,US,US-AK,King Salmon,00AN,NaN,00AN,"59.093287, -156.456699"
...,...,...,...,...,...,...,...,...,...,...,...,...
75843,ZZ-0001,heliport,Sealand Helipad,40.0,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"51.894444, 1.4825"
75844,ZZ-0002,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"-11.584277777799999, 47.296388888900005"
75845,ZZ-0003,small_airport,Fainting Goat Airport,690.0,NaN,US,US-TX,Blum,87TX,NaN,87TX,"32.110587, -97.356312"
75846,ZZZW,closed,Scandium City Heliport,4.0,NaN,CA,CA-YT,(Old) Scandium City,NaN,NaN,NaN,"69.355287, -138.93931"


In [4]:
df = df[df.type.isin(['medium_airport', 'large_airport'])]


In [5]:
df[df['continent'] == 'EU'][['iata_code', 'name']]


,iata_code,name
13631,JAM,Bezmer Air Base
13718,AEY,Akureyri Airport
13729,EGS,Egilsstaðir Airport
13745,HFN,Hornafjörður Airport
13749,HZK,Húsavík Airport
...,...,...
73020,NaN,Soltsy-2 Air Base
73022,NaN,Severomorsk-3 Naval Air Base
73024,NaN,Fedotovo Naval Air Base
73132,LNX,Smolensk North Airport


In [6]:
d = pd.read_csv(path+'estat_tgs00113_en.csv')
d


,DATAFLOW,LAST UPDATE,freq,tra_infr,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM,BG31,2010,644.0,NaN
1,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM,BG31,2011,642.0,NaN
2,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM,BG31,2012,642.0,NaN
3,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM,BG31,2013,648.0,NaN
4,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM,BG31,2014,648.0,NaN
...,...,...,...,...,...,...,...,...,...
5113,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM_TKM2,UKN0,2014,24.0,NaN
5114,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM_TKM2,UKN0,2015,24.0,NaN
5115,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM_TKM2,UKN0,2016,24.0,NaN
5116,ESTAT:TGS00113(1.0),24/11/23 23:00:00,A,RL,KM_TKM2,UKN0,2017,25.0,NaN


# Making some Train connections

In [10]:
d = pd.read_csv(path+'Complete_European_Train_Routes.csv')
d.shape

(310, 2)

In [11]:

d = d.drop_duplicates()
d

,Start,End
0,London,Paris
1,Paris,Amsterdam
2,Barcelona,Madrid
3,Milan,Rome
4,Berlin,Munich
...,...,...
200,London,York
201,Rome,Ancona
202,Madrid,Valladolid
203,Milan,Trento


In [12]:
# Creating a new column 'Route' that contains a sorted tuple of 'Start' and 'End'
d['Route'] = d.apply(lambda x: tuple(sorted([x['Start'], x['End']])), axis=1)

# Counting the occurrences of each route
route_counts = d['Route'].value_counts()

# Finding permutations: routes that appear more than once
permutations = route_counts[route_counts > 1]
print(permutations)


(Frankfurt, Munich)      2
(Munich, Vienna)         2
(Amsterdam, Brussels)    2
(Brussels, London)       2
(Geneva, Zurich)         2
(Copenhagen, Hamburg)    2
(Berlin, Frankfurt)      2
(Prague, Vienna)         2
(Berlin, Hamburg)        2
(Berlin, Copenhagen)     2
(Amsterdam, Paris)       2
(Lisbon, Madrid)         2
(Brussels, Paris)        2
(Berlin, Warsaw)         2
(Cologne, Frankfurt)     2
(Budapest, Vienna)       2
(Milan, Rome)            2
(Barcelona, Madrid)      2
Name: Route, dtype: int64


In [13]:
# Creating a new column 'Route' that contains a sorted tuple of 'Start' and 'End'
d['Route'] = d.apply(lambda x: tuple(sorted([x['Start'], x['End']])), axis=1)

# Dropping duplicates based on the 'Route' column
df_unique = d.drop_duplicates(subset='Route')

# Optionally drop the 'Route' column if it's no longer needed
df_unique = df_unique.drop(columns=['Route'])

print(df_unique)

         Start         End
0       London       Paris
1        Paris   Amsterdam
2    Barcelona      Madrid
3        Milan        Rome
4       Berlin      Munich
..         ...         ...
200     London        York
201       Rome      Ancona
202     Madrid  Valladolid
203      Milan      Trento
204  Amsterdam   The Hague

[183 rows x 2 columns]


In [14]:
df_unique.to_csv(path+'TRAIN_FINAL.csv')

In [17]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [18]:
geolocator = Nominatim(user_agent="geoapiExercises")

# To prevent spamming the API with too many requests, use a RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to fetch coordinates
def get_lat_lon(city):
    location = geocode(city)
    return (location.latitude, location.longitude) if location else (None, None)

# Apply the function to 'Start' and 'End' columns
df_unique['Start_Lat'], df_unique['Start_Lon'] = zip(*df_unique['Start'].apply(get_lat_lon))
df_unique['End_Lat'], df_unique['End_Lon'] = zip(*df_unique['End'].apply(get_lat_lon))

print(df_unique)

         Start         End  Start_Lat  Start_Lon    End_Lat    End_Lon
0       London       Paris  51.489334  -0.144055  48.858890   2.320041
1        Paris   Amsterdam  48.858890   2.320041  52.373080   4.892453
2    Barcelona      Madrid  41.382894   2.177432  40.416705  -3.703582
3        Milan        Rome  45.464194   9.189635  41.893320  12.482932
4       Berlin      Munich  52.517037  13.388860  48.137108  11.575382
..         ...         ...        ...        ...        ...        ...
200     London        York  51.489334  -0.144055  53.959055  -1.081536
201       Rome      Ancona  41.893320  12.482932  43.480120  13.218791
202     Madrid  Valladolid  40.416705  -3.703582  41.652133  -4.728562
203      Milan      Trento  45.464194   9.189635  46.066423  11.125760
204  Amsterdam   The Hague  52.373080   4.892453  52.079984   4.311346

[183 rows x 6 columns]


In [23]:
def get_lat_lon(city):
    location = geocode(city)
    return (location.latitude, location.longitude) if location else (None, None)


def calculate_distance(row):
    if None not in [row['Start_Lat'], row['Start_Lon'], row['End_Lat'], row['End_Lon']]:
        start_coords = (row['Start_Lat'], row['Start_Lon'])
        end_coords = (row['End_Lat'], row['End_Lon'])
        return geodesic(start_coords, end_coords).kilometers
    else:
        return None
    


# Calculate distances and add to a new column
df_unique['Distance_km'] = df_unique.apply(calculate_distance, axis=1)
df_unique


,Start,End,Start_Lat,Start_Lon,End_Lat,End_Lon,Distance_km
0,London,Paris,51.489334,-0.144055,48.858890,2.320041,341.412830
1,Paris,Amsterdam,48.858890,2.320041,52.373080,4.892453,431.169066
2,Barcelona,Madrid,41.382894,2.177432,40.416705,-3.703582,506.922016
3,Milan,Rome,45.464194,9.189635,41.893320,12.482932,477.322179
4,Berlin,Munich,52.517037,13.388860,48.137108,11.575382,503.981695
...,...,...,...,...,...,...,...
200,London,York,51.489334,-0.144055,53.959055,-1.081536,282.030791
201,Rome,Ancona,41.893320,12.482932,43.480120,13.218791,186.300703
202,Madrid,Valladolid,40.416705,-3.703582,41.652133,-4.728562,162.024232
203,Milan,Trento,45.464194,9.189635,46.066423,11.125760,164.809697


In [28]:
df_unique['travel_time'] = np.int64(df_unique.Distance_km / 100)
df_unique['travel_time'][df_unique['travel_time']==0] = 1
df_unique

/var/folders/k0/pdpm_bns0zb_5lqb4931xd7r0000gn/T/ipykernel_76368/189582845.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['travel_time'][df_unique['travel_time']==0] = 1


,Start,End,Start_Lat,Start_Lon,End_Lat,End_Lon,Distance_km,travel_time
0,London,Paris,51.489334,-0.144055,48.858890,2.320041,341.412830,3
1,Paris,Amsterdam,48.858890,2.320041,52.373080,4.892453,431.169066,4
2,Barcelona,Madrid,41.382894,2.177432,40.416705,-3.703582,506.922016,5
3,Milan,Rome,45.464194,9.189635,41.893320,12.482932,477.322179,4
4,Berlin,Munich,52.517037,13.388860,48.137108,11.575382,503.981695,5
...,...,...,...,...,...,...,...,...
200,London,York,51.489334,-0.144055,53.959055,-1.081536,282.030791,2
201,Rome,Ancona,41.893320,12.482932,43.480120,13.218791,186.300703,1
202,Madrid,Valladolid,40.416705,-3.703582,41.652133,-4.728562,162.024232,1
203,Milan,Trento,45.464194,9.189635,46.066423,11.125760,164.809697,1


In [19]:
df_unique.to_csv(path+'TRAIN_FINAL.csv')

In [26]:
df_unique

,Start,End,Start_Lat,Start_Lon,End_Lat,End_Lon,Distance_km
0,London,Paris,51.489334,-0.144055,48.858890,2.320041,341.412830
1,Paris,Amsterdam,48.858890,2.320041,52.373080,4.892453,431.169066
2,Barcelona,Madrid,41.382894,2.177432,40.416705,-3.703582,506.922016
3,Milan,Rome,45.464194,9.189635,41.893320,12.482932,477.322179
4,Berlin,Munich,52.517037,13.388860,48.137108,11.575382,503.981695
...,...,...,...,...,...,...,...
200,London,York,51.489334,-0.144055,53.959055,-1.081536,282.030791
201,Rome,Ancona,41.893320,12.482932,43.480120,13.218791,186.300703
202,Madrid,Valladolid,40.416705,-3.703582,41.652133,-4.728562,162.024232
203,Milan,Trento,45.464194,9.189635,46.066423,11.125760,164.809697


In [29]:
df_unique.to_csv(path+'TRAIN_FINAL.csv')